In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import warnings
import os


class DataAnalyzerApp:
    def __init__(self):
        self.file_path = None
        self.df = None
        self.processed_df = None
        self.columns_to_analyze = None
        self.modified_columns = {
            'mean': {'cols': set(), 'affected': 0},
            'median': {'cols': set(), 'affected': 0},
            'mode': {'cols': set(), 'affected': 0},
            'specific_value': {'cols': set(), 'affected': 0},
            'dropna': {'cols': set(), 'affected': 0}
        }
        self.kmeans_model = None

    def select_file(self):
        
        self.file_path = input("Introduce la ruta del archivo Excel (con extensión .xlsx): ")
        self.load_data()

    def load_data(self):
        try:
            self.df = pd.read_csv('creditcardcsvpresent.csv')
            print(f"Archivo cargado correctamente desde: {self.file_path}")
            self.select_columns_for_analysis()
        except Exception as e:
            print(f"Error al cargar el archivo: {e}")

    def select_columns_for_analysis(self):
        print("\nSelecciona las columnas que NO serán parte del análisis:")
        columns = list(self.df.columns)
        for i, col in enumerate(columns):
            print(f"{i+1}. {col}")

        selected_indices = input("Introduce los números de las columnas separadas por comas (por ejemplo: 1,3,5): ")
        if selected_indices:
            indices_to_exclude = [int(idx) - 1 for idx in selected_indices.split(",") if idx.isdigit()]
            self.columns_to_analyze = [col for i, col in enumerate(columns) if i not in indices_to_exclude]
        else:
            self.columns_to_analyze = columns

        self.df = self.df[self.columns_to_analyze]
        print(f"Las columnas que se analizarán son: {', '.join(self.columns_to_analyze)}")

    def preview_data_menu(self):
        while True:
            print("\nMenú de Previsualización:")
            print("1. Previsualizar datos")
            print("2. Estadísticos")
            print("3. Listar columnas categóricas")
            print("4. Regresar al menú principal")
            choice = input("Selecciona una opción: ")

            if choice == '1':
                self.show_data_preview()
            elif choice == '2':
                self.show_statistics()
            elif choice == '3':
                self.list_categorical_columns()
            elif choice == '4':
                break
            else:
                print("Opción no válida. Inténtalo de nuevo.")

    def show_data_preview(self):
        if self.df is not None:
            num_records = int(input("¿Cuántos registros deseas mostrar?: "))
            print(self.df.head(num_records))
        else:
            print("No hay datos cargados.")

    def show_statistics(self):
        if self.df is not None:
            numeric_cols = self.df.select_dtypes(include=[np.number]).columns
            print("Columnas numéricas disponibles:")
            for i, col in enumerate(numeric_cols):
                print(f"{i+1}. {col}")
            col_idx = int(input("Selecciona una columna por número: ")) - 1
            if 0 <= col_idx < len(numeric_cols):
                col = numeric_cols[col_idx]
                self.print_column_statistics(col)
            else:
                print("Número de columna no válido.")
        else:
            print("No hay datos cargados.")

    def print_column_statistics(self, col):
        data = self.df[col]
        print(f"\nEstadísticos para la columna '{col}':")
        print(f"Máximo: {data.max()}")
        print(f"Mínimo: {data.min()}")
        print(f"Media: {data.mean()}")
        print(f"Mediana: {data.median()}")

        mode_series = data.mode()
        if not mode_series.empty:
            print(f"Moda: {mode_series.values[0]}")
        else:
            print("Moda: No disponible")

        print(f"Primer cuartil (Q1): {data.quantile(0.25)}")
        print(f"Tercer cuartil (Q3): {data.quantile(0.75)}")

    def list_categorical_columns(self):
        if self.df is not None:
            cat_cols = self.df.select_dtypes(include=[object]).columns
            if len(cat_cols) == 0:
                print("No hay columnas categóricas en el conjunto de datos.")
                return
            print("Columnas categóricas disponibles:")
            for i, col in enumerate(cat_cols):
                print(f"{i+1}. {col}")
            col_idx = int(input("Selecciona una columna por número: ")) - 1
            if 0 <= col_idx < len(cat_cols):
                col = cat_cols[col_idx]
                self.print_column_categories(col)
            else:
                print("Número de columna no válido.")
        else:
            print("No hay datos cargados.")

    def print_column_categories(self, col):
        print(f"\nCategorías y frecuencias para la columna '{col}':")
        frequencies = self.df[col].value_counts()
        for category, freq in frequencies.items():
            print(f"{category}: {freq}")

    def process_data(self):
        if self.df is None:
            print("No hay datos cargados.")
            return

        while True:
            print("\nMenú de Procesamiento de Datos:")
            print("1. Missing")
            print("2. Clasificación")
            print("3. Anomalías")
            print("4. Resumen")
            print("5. Regresar al menú principal")
            choice = input("Selecciona una opción: ")

            if choice == '1':
                self.handle_missing_values()
            elif choice == '2':
                self.classification()
            elif choice == '3':
                self.anomalies()
            elif choice == '4':
                self.summary()
            elif choice == '5':
                break
            else:
                print("Opción no válida. Inténtalo de nuevo.")

    def classification(self):
        if self.df is None:
            print("No hay datos cargados.")
            return

        num_clusters = int(input("Introduce el número de clusters (K) para K-means: "))
        numeric_cols = self.df.select_dtypes(include=[np.number]).columns
        if numeric_cols.empty:
            print("No hay columnas numéricas para aplicar K-means.")
            return

        # Crear un DataFrame de entrada para KMeans sin afectar el DataFrame original
        data_for_clustering = self.df[numeric_cols].dropna()
        kmeans = KMeans(n_clusters=num_clusters, n_init=10, random_state=42)
        clusters = kmeans.fit_predict(data_for_clustering)

        # Asegúrate de que el número de clusters coincide con el número de registros
        if len(clusters) == len(self.df):
            self.df.loc[data_for_clustering.index, 'Cluster'] = clusters
        else:
            print(f"Error: El número de registros en el DataFrame de clustering ({len(clusters)}) no coincide con el DataFrame original ({len(self.df)}).")

        self.kmeans_model = kmeans
        print(f"K-means ha sido aplicado con {num_clusters} clusters.")
        
        # Imprimir los centroides
        print("\nCentroides encontrados:")
        for i, centroid in enumerate(kmeans.cluster_centers_):
            print(f"Cluster {i}: {centroid}")

        # Mostrar gráfico de los clusters
        self.plot_clusters(data_for_clustering)

    def plot_clusters(self, data):
        # Reducción de dimensionalidad a 3D para la visualización
        pca = PCA(n_components=3)
        data_pca = pca.fit_transform(data)
        centroids_pca = pca.transform(self.kmeans_model.cluster_centers_)

        fig = plt.figure(figsize=(12, 10))
        ax = fig.add_subplot(111, projection='3d')

        # Obtén los clusters del DataFrame original
        clusters = self.df.loc[data.index, 'Cluster'].values
        
        scatter = ax.scatter(data_pca[:, 0], data_pca[:, 1], data_pca[:, 2], c=clusters, cmap='viridis', marker='o')
        
        # Dibujar los centroides
        ax.scatter(centroids_pca[:, 0], centroids_pca[:, 1], centroids_pca[:, 2], s=200, c='red', marker='x', label='Centroides')

        legend1 = ax.legend(*scatter.legend_elements(), title="Clusters")
        ax.add_artist(legend1)
        ax.set_title('Visualización de Clusters K-means en 3D')
        ax.set_xlabel('Componente Principal 1')
        ax.set_ylabel('Componente Principal 2')
        ax.set_zlabel('Componente Principal 3')
        plt.show()


    def anomalies(self):
        if self.df is not None:
            numeric_cols = self.df.select_dtypes(include=[np.number]).columns
            if numeric_cols.empty:
                print("No hay columnas numéricas para la detección de anomalías.")
                return

            print("Columnas numéricas disponibles:")
            for i, col in enumerate(numeric_cols):
                print(f"{i+1}. {col}")

            col_idx = int(input("Selecciona una columna por número: ")) - 1
            if 0 <= col_idx < len(numeric_cols):
                col = numeric_cols[col_idx]
                k = float(input("Introduce el valor de K para el rango intercuartílico (IQR) o presiona Enter para usar 1.5: ") or 1.5)
                self.detect_anomalies(col, k)
            else:
                print("Número de columna no válido.")
        else:
            print("No hay datos cargados.")

    def detect_anomalies(self, col, k):
        Q1 = self.df[col].quantile(0.25)
        Q3 = self.df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - k * IQR
        upper_bound = Q3 + k * IQR
        anomalies = self.df[(self.df[col] < lower_bound) | (self.df[col] > upper_bound)]
        print(f"Número de anomalías detectadas en la columna '{col}': {len(anomalies)}")

    def handle_missing_values(self):
        if self.df is not None:
            print("\nOpciones de imputación:")
            print("1. Imputar con la media")
            print("2. Imputar con la mediana")
            print("3. Imputar con la moda")
            print("4. Imputar con un valor específico")
            print("5. Eliminar filas con valores faltantes")
            choice = input("Selecciona una opción: ")

            if choice in ['1', '2', '3', '4', '5']:
                if choice != '5':
                    print("Columnas numéricas disponibles:")
                    for i, col in enumerate(self.df.select_dtypes(include=[np.number]).columns):
                        print(f"{i+1}. {col}")
                    col_idx = int(input("Selecciona una columna por número: ")) - 1
                    if 0 <= col_idx < len(self.df.select_dtypes(include=[np.number]).columns):
                        col = self.df.select_dtypes(include=[np.number]).columns[col_idx]
                        self.impute_missing_values(col)
                    else:
                        print("Número de columna no válido.")
                else:
                    self.drop_missing_rows()
            else:
                print("Opción no válida. Imputación cancelada.")
        else:
            print("No hay datos cargados.")

    def impute_missing_values(self, col):
        method = input("Selecciona el método de imputación (media/mediana/moda/valor específico): ").lower()
        if method == 'media':
            mean_value = self.df[col].mean()
            self.df[col].fillna(mean_value, inplace=True)
            self.modified_columns['mean']['cols'].add(col)
            self.modified_columns['mean']['affected'] += self.df[col].isnull().sum()
            print(f"Valores faltantes imputados con la media ({mean_value}) en la columna '{col}'.")

        elif method == 'mediana':
            median_value = self.df[col].median()
            self.df[col].fillna(median_value, inplace=True)
            self.modified_columns['median']['cols'].add(col)
            self.modified_columns['median']['affected'] += self.df[col].isnull().sum()
            print(f"Valores faltantes imputados con la mediana ({median_value}) en la columna '{col}'.")

        elif method == 'moda':
            mode_value = self.df[col].mode()[0]
            self.df[col].fillna(mode_value, inplace=True)
            self.modified_columns['mode']['cols'].add(col)
            self.modified_columns['mode']['affected'] += self.df[col].isnull().sum()
            print(f"Valores faltantes imputados con la moda ({mode_value}) en la columna '{col}'.")

        elif method == 'valor específico':
            specific_value = float(input("Introduce el valor específico para imputar: "))
            self.df[col].fillna(specific_value, inplace=True)
            self.modified_columns['specific_value']['cols'].add(col)
            self.modified_columns['specific_value']['affected'] += self.df[col].isnull().sum()
            print(f"Valores faltantes imputados con el valor específico ({specific_value}) en la columna '{col}'.")

        else:
            print("Método no válido.")

    def drop_missing_rows(self):
        affected_before = self.df.isnull().sum().sum()
        self.df.dropna(inplace=True)
        affected_after = self.df.isnull().sum().sum()
        affected_rows = affected_before - affected_after
        self.modified_columns['dropna']['affected'] += affected_rows
        print(f"Filas con valores faltantes eliminadas. Registros afectados: {affected_rows}")

    def summary(self):
        if self.modified_columns:
            print("\nResumen de modificaciones realizadas:")
            for method, info in self.modified_columns.items():
                if info['cols']:
                    print(f"\nColumnas afectadas por {method}:")
                    for i, col in enumerate(info['cols'], 1):
                        print(f"  {i}. {col}")
                    print(f"Número total de registros afectados por {method}: {info['affected']}")
        else:
            print("No se han realizado operaciones de imputación o eliminación de valores faltantes.")

    def show_correlation_matrix(self):
        if self.df is not None:
            plt.figure(figsize=(10, 8))
            sns.heatmap(self.df.select_dtypes(include=[np.number]).corr(), annot=True, cmap="coolwarm", vmin=-1, vmax=1)
            plt.title("Matriz de correlación")
            plt.show()
        else:
            print("No hay datos cargados.")

    def show_boxplot(self, use_processed):
        df_to_plot = self.processed_df if use_processed and self.processed_df is not None else self.df
        if df_to_plot is not None:
            numeric_cols = df_to_plot.select_dtypes(include=[np.number]).columns
            if numeric_cols.empty:
                print("No hay columnas numéricas para mostrar el boxplot.")
                return

            print("Columnas numéricas disponibles:")
            for i, col in enumerate(numeric_cols):
                print(f"{i+1}. {col}")

            col_idx = int(input("Selecciona una columna por número: ")) - 1
            if 0 <= col_idx < len(numeric_cols):
                col = numeric_cols[col_idx]
                plt.figure(figsize=(8, 6))
                sns.boxplot(df_to_plot[col])
                plt.title(f"Boxplot de la columna '{col}'")
                plt.show()
            else:
                print("Número de columna no válido.")
        else:
            print("No hay datos cargados.")

    def show_histogram(self):
        if self.df is not None:
            use_processed = input("¿Deseas trabajar con los datos procesados? (sí/no): ").lower() == 'sí'
            df_to_plot = self.processed_df if use_processed and self.processed_df is not None else self.df

            numeric_cols = df_to_plot.select_dtypes(include=[np.number]).columns
            if numeric_cols.empty:
                print("No hay columnas numéricas para mostrar el histograma.")
                return

            print("Columnas numéricas disponibles:")
            for i, col in enumerate(numeric_cols):
                print(f"{i+1}. {col}")

            col_idx = int(input("Selecciona una columna por número: ")) - 1
            if 0 <= col_idx < len(numeric_cols):
                col = numeric_cols[col_idx]
                plt.figure(figsize=(8, 6))
                df_to_plot[col].hist(bins=30)
                plt.title(f"Histograma de la columna '{col}'")
                plt.xlabel(col)
                plt.ylabel("Frecuencia")
                plt.show()
            else:
                print("Número de columna no válido.")
        else:
            print("No hay datos cargados.")

    def menu(self):
        while True:
            print("\nMenú Principal:")
            print("1. Seleccionar archivo")
            print("2. Visualización de datos")
            print("3. Procesar datos")
            print("4. Mostrar matriz de correlación")
            print("5. Mostrar boxplot")
            print("6. Mostrar histograma")
            print("7. Salir")
            choice = input("Selecciona una opción: ")

            if choice == '1':
                self.select_file()
            elif choice == '2':
                self.preview_data_menu()
            elif choice == '3':
                self.process_data()
            elif choice == '4':
                self.show_correlation_matrix()
            elif choice == '5':
                processed = input("¿Deseas usar los datos procesados (sí/no)?: ").lower() == 'sí'
                self.show_boxplot(processed)
            elif choice == '6':
                self.show_histogram()
            elif choice == '7':
                print("Saliendo del programa...")
                break
            else:
                print("Opción no válida. Inténtalo de nuevo.")

if __name__ == "__main__":
    app = DataAnalyzerApp()
    app.menu()



Menú Principal:
1. Seleccionar archivo
2. Visualización de datos
3. Procesar datos
4. Mostrar matriz de correlación
5. Mostrar boxplot
6. Mostrar histograma
7. Salir


Selecciona una opción:  1
Introduce la ruta del archivo Excel (con extensión .xlsx):  2


Archivo cargado correctamente desde: 2

Selecciona las columnas que NO serán parte del análisis:
1. Merchant_id
2. Transaction date
3. Average Amount/transaction/day
4. Transaction_amount
5. Is declined
6. Total Number of declines/day
7. isForeignTransaction
8. isHighRiskCountry
9. Daily_chargeback_avg_amt
10. 6_month_avg_chbk_amt
11. 6-month_chbk_freq
12. isFradulent


Introduce los números de las columnas separadas por comas (por ejemplo: 1,3,5):  2


Las columnas que se analizarán son: Merchant_id, Average Amount/transaction/day, Transaction_amount, Is declined, Total Number of declines/day, isForeignTransaction, isHighRiskCountry, Daily_chargeback_avg_amt, 6_month_avg_chbk_amt, 6-month_chbk_freq, isFradulent

Menú Principal:
1. Seleccionar archivo
2. Visualización de datos
3. Procesar datos
4. Mostrar matriz de correlación
5. Mostrar boxplot
6. Mostrar histograma
7. Salir


...

Columnas categóricas disponibles:
1. Is declined
2. isForeignTransaction
3. isHighRiskCountry
4. isFradulent


Selecciona una columna por número:  1



Categorías y frecuencias para la columna 'Is declined':
N: 3018
Y: 57

Menú de Previsualización:
1. Previsualizar datos
2. Estadísticos
3. Listar columnas categóricas
4. Regresar al menú principal


Selecciona una opción:  1
¿Cuántos registros deseas mostrar?:  3000


      Merchant_id  ...  isFradulent
0      3160040998  ...            Y
1      3160040998  ...            Y
2      3160041896  ...            Y
3      3160141996  ...            Y
4      3160241992  ...            Y
...           ...  ...          ...
2995   6571515078  ...            N
2996   6572732555  ...            N
2997   6574569646  ...            N
2998   6575815976  ...            N
2999   6576529686  ...            N

[3000 rows x 11 columns]

Menú de Previsualización:
1. Previsualizar datos
2. Estadísticos
3. Listar columnas categóricas
4. Regresar al menú principal
